In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os

In [3]:
os.chdir('/content/gdrive/My Drive')

In [4]:
file_path = os.getcwd() + '/1'

In [5]:
# file_ = open(file_path , 'r' , encoding='iso-8859-1')
# 
# data = file_.readlines()

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt


from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply , Embedding
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

# from faker import Faker
import random
from tqdm import tqdm
# from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline



Using TensorFlow backend.


In [7]:
import tensorflow as tf
import tensorflow_addons as tfa
print(tf.__version__)
from sklearn.model_selection import train_test_split
import os
import io
import numpy as np
import re
import unicodedata
import urllib3
import shutil
import zipfile
import itertools
from string import digits

2.2.0


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
def preprocess_sentence(sentence):
    
    num_digits= str.maketrans('','', digits)
    
    sentence= sentence.lower()
    sentence= re.sub(" +", " ", sentence)
    sentence= re.sub("'", '', sentence)
    sentence= sentence.translate(num_digits)
    sentence= re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = sentence.rstrip().strip()
    sentence = "<start> " + sentence + " <end>"
    
    return sentence


In [10]:

text = []
count = 0


with open(file_path ,'r' , encoding='iso-8859-1') as txtfile:
  for line in txtfile.readlines():
    if count == 20000:
      break

    text.append(preprocess_sentence(line.split('+++$+++')[-1]))
    count +=1
    
    


In [11]:
len(text)

20000

In [12]:
text[0]

'<start> they do not ! <end>'

In [13]:

max_len = 20
trunc_token = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(oov_token=oov_tok , filters = "")
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(text)
padded = pad_sequences(sequences,maxlen=max_len, truncating=trunc_token,padding = 'post')


In [14]:
len(word_index)

13051

In [15]:
padded[10008]

array([   3,   95,    6,   43,   11, 2168,   91,  114,    7,    4,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [16]:
index_word = {}
for word , index in word_index.items():
    index_word[index] = word
    
    

In [17]:
def create_word_embeddings(file_path):
    
    with open(file_path , 'r') as f:
        wordToEmbedding = {}
        wordToIndex = {}
        indexToWord = {}
        
        for line in f:
            data = line.strip().split()
            token = data[0]
            wordToEmbedding[token] = np.array(data[1:] ,dtype = np.float64)



        
        tokens = sorted(wordToEmbedding.keys())
        for idx , token in enumerate(tokens):
            idx = idx + 1 #for zero masking
            wordToIndex[token] = idx
            indexToWord[idx] = token

    return wordToEmbedding , wordToIndex , indexToWord
            
            

In [18]:
wordToEmbedding , wordToIndex , indexToWord = create_word_embeddings(os.getcwd()+ '/glove.6B.200d.txt')

In [19]:
len(wordToIndex)    

400000

In [20]:
def create_pretrained_embedding_layer(wordToEmbedding , wordToIndex , indexToWord):
    
    vocablen = len(word_index)+1 #for zero masking
    embedding_dimensions = 200
    
    embeddingMatrix = np.zeros((vocablen , embedding_dimensions))
    count = 0
    for word , index in word_index.items():
        if word not in wordToEmbedding.keys():
            embeddingMatrix[index ,:] = np.random.uniform(low = -1 , high =1 ,size = (1,200))
            count +=1
        else :
            
            embeddingMatrix[index , :] = wordToEmbedding[word]
        
    embeddingLayer = Embedding(vocablen , embedding_dimensions , weights = [embeddingMatrix] , trainable = False)
    print(embeddingMatrix.shape)
    print(count)
    
    return embeddingMatrix

In [21]:
len(word_index)

13051

In [22]:
embeddingMatrix = create_pretrained_embedding_layer(wordToEmbedding , wordToIndex , indexToWord)


(13052, 200)
1601


In [23]:
embeddingMatrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.09444563,  0.67830645, -0.23497883, ..., -0.33542104,
        -0.91926708, -0.63332775],
       [ 0.12289   ,  0.58037   , -0.069635  , ..., -0.039174  ,
        -0.16236   , -0.096652  ],
       ...,
       [-0.25924   , -0.81193   ,  0.45847   , ...,  0.045301  ,
         0.01518   , -0.36259   ],
       [ 0.53246   ,  0.84359   ,  0.13337   , ...,  0.11501   ,
        -0.33268   , -0.55189   ],
       [-0.018711  ,  0.39573   ,  0.84356   , ...,  0.31973   ,
        -0.28322   , -0.12453   ]])

In [36]:
def get_dataset(padded):
    index = 0
    context = np.zeros((500,max_len))
    response = np.zeros((500,max_len))
    for idx in range(2000,3000,2):
        context[index,:] = padded[idx]
        response[index,:] = padded[idx+1]
        index +=1
    return context , response

In [37]:
context , response = get_dataset(padded)

response1 = response[:,:-1]
response2 = response[:,1:]

In [38]:
response2 = list(response2.swapaxes(0,1))

In [39]:
import tensorflow as tf
from keras.layers import Layer
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt


from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply , Embedding , Dense , TimeDistributed
from keras.layers import RepeatVector, Dense, Activation, Lambda , LSTMCell , Reshape , Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model

class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

class seq2seqmodel():

  def __init__(self, Tx, Ty, rnn_hidden_units, embedding_dims, 
                 vocab_size, embedding_matrix):
      
    self.Tx = Tx
    self.Ty = Ty
    self.rnn_hidden_units = rnn_hidden_units
    self.embedding_dims = embedding_dims
    self.vocab_size = vocab_size
    self.embedding_matrix = embedding_matrix
    self.batch_size = batch_size
    self.attention_layer = attention()
    self.dense_layer = Dense(10 , activation = 'tanh')
    self.post_lstm_layer = LSTM(self.rnn_hidden_units, return_state = True)
    self.repeator = RepeatVector(self.Tx)
    self.concatenator = Concatenate(axis=-1)
    self.ouput_layer = Dense(self.vocab_size , activation = 'softmax')

  def build(self,
            input_shape = None,
            output_shape = None):
    self.input_shape = input_shape
    self.ouput_shape = output_shape


    encoder_input = Input(shape = (self.Tx,) , name = "encoder_input")
    decoder_input = Input(shape = (self.Ty,) , name = "decoder_input")
    S0 = Input(shape = (self.rnn_hidden_units ,) , name ="S0")
    C0 = Input(shape = (self.rnn_hidden_units ,) , name = "C0")

    encoder_embeddings = Embedding(input_dim = self.vocab_size ,
                                  output_dim = self.embedding_dims,
                                  weights = [self.embedding_matrix], 
                                  trainable = False)(encoder_input)
    
    decoder_embeddings = Embedding(input_dim = self.vocab_size,
                                  output_dim = self.embedding_dims,
                                  weights = [self.embedding_matrix], 
                                  trainable = False)(decoder_input)
    
    activations,last_hidden_state,last_cell_state = (LSTM(units = self.rnn_hidden_units,
                                                                return_sequences = True,
                                                                return_state = True))(encoder_embeddings)

    s = last_hidden_state
    c = last_cell_state
    outputs = []
    
    for t in range(self.Ty):

      repeator = self.repeator(s)
      
      concat = self.concatenator(inputs = [activations, repeator])
      
      context = self.attention_layer(inputs = concat)

      d_embeddings = Lambda(lambda x: x[:,t,:])(decoder_embeddings)

      decoder = self.concatenator(inputs = [context , d_embeddings ])

      
      decoder = Reshape(target_shape = (1 , 712))(decoder)

      s, _ , c = self.post_lstm_layer(inputs = decoder , initial_state = [s ,c])

      out = self.ouput_layer(s)

      outputs.append(out)

    
    model = Model(inputs = [encoder_input , decoder_input , S0 , C0] , outputs = outputs )

    return model
# --------------- model_parameters-----------------------------
Tx = 20
Ty = 19
rnn_hidden_units = 256
embedding_dims = 200
vocab_size = len(word_index) + 1
embedding_matrix = embeddingMatrix
dense_units = 256
batch_size = 500
attention_layer = attention()

In [41]:
model = seq2seqmodel(Tx, Ty, rnn_hidden_units, embedding_dims, 
                 vocab_size, embedding_matrix) 
model =model.build()
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 20, 200)      2610400     encoder_input[0][0]              
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, 20, 256), (N 467968      embedding_10[0][0]               
__________________________________________________________________________________________________
repeat_vector_5 (RepeatVector)  (None, 20, 256)      0           lstm_10[0][1]                    
                                                                 lstm_9[0][0]               

In [42]:
opt = Adam(lr = 0.005, beta_1 = 0.9,beta_2 = 0.999 , decay = 0.01)
model.compile(optimizer = opt , loss = "sparse_categorical_crossentropy" ,metrics = ['accuracy'])

In [43]:
model.fit([context , response1, np.zeros((batch_size , rnn_hidden_units)) , np.zeros((batch_size , rnn_hidden_units))] , response2 , epochs = 100 , batch_size = 10)

Epoch 1/100
500/500 [==============================] - 40s 80ms/step - loss: 88.0745 - dense_10_loss: 2.6429 - dense_10_accuracy: 0.0200 - dense_10_accuracy_1: 0.0180 - dense_10_accuracy_2: 0.0140 - dense_10_accuracy_3: 0.0540 - dense_10_accuracy_4: 0.1080 - dense_10_accuracy_5: 0.1660 - dense_10_accuracy_6: 0.2460 - dense_10_accuracy_7: 0.3240 - dense_10_accuracy_8: 0.3740 - dense_10_accuracy_9: 0.4240 - dense_10_accuracy_10: 0.4640 - dense_10_accuracy_11: 0.5120 - dense_10_accuracy_12: 0.5600 - dense_10_accuracy_13: 0.5940 - dense_10_accuracy_14: 0.6380 - dense_10_accuracy_15: 0.6560 - dense_10_accuracy_16: 0.6720 - dense_10_accuracy_17: 0.6920 - dense_10_accuracy_18: 0.7360
Epoch 2/100
500/500 [==============================] - 30s 61ms/step - loss: 65.7192 - dense_10_loss: 1.5060 - dense_10_accuracy: 0.0940 - dense_10_accuracy_1: 0.1040 - dense_10_accuracy_2: 0.1400 - dense_10_accuracy_3: 0.1820 - dense_10_accuracy_4: 0.2400 - dense_10_accuracy_5: 0.2940 - dense_10_accuracy_6: 0.36

KeyboardInterrupt: ignored

In [44]:
s = ['<start> do you support congress <end>']
r= ['<start> yes i do , you should also support congress <end>']

seq = tokenizer.texts_to_sequences(s)
pad = pad_sequences(seq,maxlen=max_len, truncating=trunc_token,padding = 'post')

seq = tokenizer.texts_to_sequences(r)
pad2 = pad_sequences(seq,maxlen=max_len, truncating=trunc_token,padding = 'post')

pad2 = pad2[:,:-1]

print(pad)

pred = model.predict([pad , pad2 , np.zeros((1, rnn_hidden_units)) , np.zeros((1,rnn_hidden_units))])




[[   3   27    6 2119 8067    4    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]


In [45]:
import math
def beam_search_decoder(predictions, top_k = 3):

    output_sequences = [([], 0)]
    
    for token_probs in predictions:
        new_sequences = []
        token_probs = token_probs.reshape(len(word_index)+1 ,)
        for old_seq, old_score in output_sequences:
            for char_index in range(len(token_probs)):
                new_seq = old_seq + [char_index]
                new_score = old_score + math.log(token_probs[char_index])
                new_sequences.append((new_seq, new_score))
                
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:top_k]
        
    return output_sequences

In [46]:
ans = beam_search_decoder(pred)
print(ans)


[([8, 5, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], -21.46504867779595), ([8, 5, 5, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], -21.636926359382915), ([8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], -21.853577822239643)]
